In [79]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'sklearn'

In [44]:
df = pd.read_csv('/Users/brandon/Desktop/MILO/test_files/dummy_train_column_scan.csv')
df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN

In [81]:
valid_data_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']

valid = df.select_dtypes(include=valid_data_types)

invalid = df.drop(columns=valid.columns)
fix = pd.DataFrame()

#process mixed data types
comments = {}

for column in invalid.columns:
    col = invalid[column]

    total_count = col.shape[0]
    nan_count = col.isna().sum()

    col = col.dropna()
    not_nan_count = col.shape[0]

    numeric = ~pd.to_numeric(col, errors='coerce').isna()
    numeric_count = numeric.sum()

    if (numeric_count / total_count) > 0.6:
        invalid[column] = pd.to_numeric(col, errors='coerce')
        comments[column] = f'More than 0.6 of data is numeric. {nan_count} dropped' 
    
    elif (len(col.unique()) == 2):
        mapping = {}
        for index, val in enumerate(col.unique()):
            mapping[val] = index
        invalid[column] = col.map(mapping).astype('int')
        comments[column] = f'Two unique values. {str(mapping)}'
 
    elif (len(col.unique()) > 2):
        comments[column] = 'Will be one hot encoded'
        

print(comments)






    


 

{'va_c': 'More than 0.6 of data is numeric. 2 dropped', 'cat_a': 'Will be one hot encoded', 'cat_b': "Two unique values. {'a': 0, 'b': 1}"}
